In [14]:
%%file AlgoResource_new.py
# -*- coding:utf-8 -*-
import falcon
import json
import sys
import logging
from AIFlysdk.AIFlyApi import AIFlyApi
from model_loader import ModelLoader
import yaml

try:
    from inspect import signature, _empty
except:
    from funcsigs import signature, _empty
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('Algo Engine')
PY2 = (sys.version_info.major == 2)
PY3 = (sys.version_info.major == 3)

class PredictionInput:
    """
    Contains input details and model_id and data
    model_id:'tensormodel'
    model_version:'0.1.0
    data:
        - input:数据输入
    """  
    def __init__(self,model_id,model_version,data):
        self.model_id=model_id
        self.model_version=model_version
        self.data=data

class ModelInput:
    """
    Contains input details and model_id and data
    funcslist:['predpy.pred']
        - predpy:文件名
        - pred:函数名
    """  
    def __init__(self,funcslist):
        self.funcslist=funcslist
class AlgoResource(object):
    def __init__(self,config):
        self.logger = logging.getLogger(__name__)
        self.conf=config#'config/AIFlyapi_config.yaml'
        self.models_loaded={}
        with open(self.conf, 'rt') as f:
            self.client_config = yaml.load(f.read())
        #model_list=[['fib_model','1.0.2']]
        #model_loader = ModelLoader(self.client_config)
        #models_to_load=model_list
        self.models_loaded = {}#model_loader.get_models_from_list(models_to_load)

    def on_get(self, req, resp):
        """Handles GET requests"""
        resp.status = falcon.HTTP_200  # This is the default status
        resp.body = ('\nFalcon is awesome! \n')
    def on_put(self, req, resp):
        """Handles PUT requests for model init"""
        funcs_to_service_map = json.loads(req.stream.read())
        self.logger.debug('Update request received with payload: %s', str(funcs_to_service_map))
        model_list=funcs_to_service_map['model_list']
        model_loader = ModelLoader(self.client_config)
        models_to_load=model_list
        self.models_loaded = model_loader.get_models_from_list(models_to_load)
        #model_list=[["HelloWorldExample", "1.0.0"],["HelloAI2", "1.0.0"],["TensorflowMnistExample","1.0.0"],['fib_model',"1.0.2"]]
        
        resp.status = falcon.HTTP_200
        #resp.content=json.dumps({"reg info":"sucess"})
        resp.body = json.dumps({"Init Model Service": "Success"})
    def on_post(self, req, resp):
        payload = json.loads(req.stream.read())

        if ('data' in payload.keys()) and ('modelId' in payload.keys()):
            PredictionInput.data = payload['data']
            PredictionInput.model_id = payload['modelId']
            PredictionInput.model_version=payload['model_version']
            key = (PredictionInput.model_id, PredictionInput.model_version)
            curr_model = self.models_loaded[key]
             
        else:
            resp.status = falcon.HTTP_400
            raise falcon.HTTPBadRequest("Bad Request", "Url and(or) modelId missing in the payload")

        #result=algo_engine.call_func(fun_path,**{'n':42})
        try:
            po = json.dumps({'result':curr_model.predict(PredictionInput.data)})
            resp.status = falcon.HTTP_200
            resp.body = po
        except:
            resp.body = json.dumps({'status': 'Failure', 'message' : 'Error occurred'})
            resp.status = falcon.HTTP_500
            raise falcon.HTTPInternalServerError('Internal Server Error', 'Predict failed! ')
        '''
        if po.bo.status == 'Success':
            resp.status = falcon.HTTP_200
            resp.body = (str(po.values))
        elif po.bo.status == 'Failure':
            resp.body = json.dumps({'status': 'Failure', 'message' : 'Error occurred'})
            resp.status = falcon.HTTP_500
            raise falcon.HTTPInternalServerError('Internal Server Error', 'Predict failed! ')       
        '''


Overwriting AlgoResource_new.py


In [2]:
from model_loader import ModelLoader

import yaml
conf='config/AIFlyapi_config.yaml'
with open(conf, 'rt') as f:
    client_config = yaml.load(f.read())


model_list=[["HelloWorldExample", "1.0.0"],["HelloAI2", "1.0.0"],["TensorflowMnistExample","1.0.0"],['fib_model',"1.0.2"]]
model_loader = ModelLoader(client_config)
models_to_load=model_list
models_loaded = model_loader.get_models_from_list(models_to_load)

/Users/leepand/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11dfe95d0>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/model_resources/TensorflowMnistExample_1.0.0/', '_save_summary_steps': 100}


In [4]:
models_loaded

{('HelloAI2', '1.0.0'): <__main__.HelloWorldModel instance at 0x107f07f38>,
 ('HelloWorldExample',
  '1.0.0'): <__main__.HelloWorldModel instance at 0x107f07e60>,
 ('TensorflowMnistExample',
  '1.0.0'): <TensorflowMnistExample_1_0_0.TensorflowMnistEstimator instance at 0x107dd40e0>,
 ('fib_model', '1.0.2'): <fib_model_1_0_2.fib instance at 0x11dff7878>}

In [5]:
model_id, model_version="fib_model","1.0.2"
import json
key = (model_id, model_version)
curr_model = models_loaded[key]
data = {'n':2}
#json_payload = json.dumps(data)
#json_payload = json.loads(data)
#json_payload={'n':2}
output = curr_model.predict(2)
output

2

In [2]:
import json
from AIFlysdk.AIFlyApi import AIFlyApi
class HelloWorldModel:
    def predict(self, input):
        return json.dumps("Hello World")

model = HelloWorldModel()
print "Local Prediction:", model.predict(None)
y=AIFlyApi(config='config/AIFlyapi_config.yaml')
y.publish_model(model, "HelloWorldExample", "1.0.5")

Local Prediction: "Hello World"


In [ ]:
import os
y=AIFlyApi(config='config/AIFlyapi_config.yaml')
dir_path='/Users/leepand/Downloads/BigRLab_APIs/demoday_fs/web_lab/AIaas/AIserver/AIaasFly/AIFlysdk'
y.publish_asm_model(path_to_prediction_module=dir_path+'/model/pred/fib.py',
            path_to_model_resources_dir=dir_path+'/model/model',
            model_id="fib_model",
                    model_version="1.0.2")